# Tiny TextGrad Examples: Basic Prompt Optimization

This notebook demonstrates how to use Tiny TextGrad to optimize a prompt for a given set of inputs. The prompt is optimized using a simple text loss function that focuses on clarity, accuracy, and handling of cases where the answer is not in the text.


In [1]:
from textwrap import dedent

from tinytextgrad.llm import call_llm
from tinytextgrad.prompt import (
    optimize_prompt,
)

In [2]:
initial_prompt = dedent("""
Given some text and a question, determine if the text
contains the answer to the question
""").strip()

inputs = [
    "Text: The cat is on the mat. Question: Where is the cat?",
    "Text: The sky is blue. Question: What color is the grass?",
    "Text: Paris is the capital of France. Question: What is the capital of Germany?",
    "Text: The Earth orbits the Sun. Question: What does the Earth orbit?",
    "Text: Water freezes at 0 degrees Celsius. Question: At what temperature does water boil?",
    "Text: Beethoven composed many symphonies. Question: Who composed the Fifth Symphony?",
    "Text: Elephants are the largest land animals. Question: What is the largest land animal?",
    "Text: Shakespeare wrote 'Romeo and Juliet.' Question: Who wrote 'Hamlet'?",
    "Text: Humans have 206 bones. Question: How many bones do humans have?",
    "Text: Coffee is typically grown in tropical regions. Question: Where is coffee typically grown?",
]

result = optimize_prompt(
    initial_prompt,
    "gpt-3.5-turbo",
    "gpt-4o",
    inputs,
    num_iterations=1,
)

print("\n\nFinal optimized TEXT_CONTAINS_ANSWER_PROMPT:")
print(result.variable.value)

TEXT_CONTAINS_ANSWER_PROMPT = result

2024-07-12 05:03:38.885 | DEBUG    | tinytextgrad.ttg:optimize_text:165 - ∇ 1. Current prompt:

Given some text and a question, determine if the text
contains the answer to the question
2024-07-12 05:03:54.248 | DEBUG    | tinytextgrad.ttg:step:152 - ∇ Loss:

### Prompt Evaluation

#### Clarity
The prompt is relatively clear in its instruction to determine if the given text contains the answer to the specified question. However, it could be slightly more explicit in stating that the response should be either "Yes" or "No, the text does not contain the answer to the question."

#### Specificity
The prompt provides necessary details without being overly verbose, but it could benefit from more explicit guidance on the expected format of the response.

#### Relevance
All the information included in the prompt is pertinent to the task of assessing whether the text contains the answer to the question.

#### Consistency
The prompt does not contain any contradictory instructions. However, the 



Final optimized TEXT_CONTAINS_ANSWER_PROMPT:
Given some text and a question, determine if the text contains the answer. Respond with either "Yes" or "No, the text does not contain the answer."


In [3]:
for _input in inputs:
    response = call_llm(
        model="gpt-3.5-turbo",
        prompt=TEXT_CONTAINS_ANSWER_PROMPT.variable.value,
        prompt_input=_input,
        temperature=0.7,
        max_tokens=100,
        top_p=0.95,
        frequency_penalty=0,
    )
    print(response)

Yes
No, the text does not contain the answer.
No, the text does not contain the answer.
Yes
No, the text does not contain the answer.
Yes
Yes
No, the text does not contain the answer.
Yes
Yes
